In [1]:
# 基于 https://til.simonwillison.net/llms/python-react-pattern

import openai
import re
import httpx
import os
from dotenv import load_dotenv
from openai import OpenAI

# 从 .env 文件加载环境变量
load_dotenv()

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "你好"}],
    temperature=1.0,
    top_p=1.0,
    model=model
)

chat_completion.choices[0].message.content

'你好！有什么我可以帮您的吗？ 😊'

## Agent 类解释

这个 Agent 类是一个简单的聊天机器人实现，它封装了与 OpenAI API 的交互。让我们逐个分析它的组件：

**初始化方法 (`__init__`)**：
- 接受一个可选的 `system` 参数，用于设置系统提示词
- 初始化一个空的 `messages` 列表来存储对话历史
- 如果提供了系统提示词，会将其作为第一条消息添加到对话历史中，角色为 "system"

**调用方法 (`__call__`)**：
- 使 Agent 实例可以像函数一样被调用
- 将用户输入的消息添加到对话历史中，角色为 "user"
- 调用 `execute()` 方法获取AI响应
- 将AI的响应添加到对话历史中，角色为 "assistant"
- 返回AI的响应内容

**执行方法 (`execute`)**：
- 使用 OpenAI 客户端创建聊天完成请求
- 设置温度为0以获得更确定性的响应
- 传递完整的对话历史给API
- 返回AI响应的文本内容

这种设计模式允许Agent维护对话上下文，使得多轮对话成为可能。每次调用都会保存对话历史，让AI能够理解之前的交互内容。

In [21]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model=model, 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [22]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [23]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [24]:
abot = Agent(prompt)

result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using average_dog_weight.
Action: average_dog_weight: Toy Poodle
PAUSE


In [25]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [26]:
next_prompt = "Observation: {}".format(result)

In [27]:
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [28]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [29]:
abot = Agent(prompt)

question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [30]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [31]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [32]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [33]:
abot(next_prompt)

'Thought: I now have the average weights for both dogs. I will add 37 lbs (Border Collie) and 20 lbs (Scottish Terrier) to get their combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [34]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [35]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### 添加循环

### 正则表达式模式，用于识别和解析AI助手响应中的动作指令。模式解析：

- ^ - 行的开始
- Action:  - 字面匹配"Action: "
- (\w+) - 第一个捕获组，匹配一个或多个单词字符（动作名称）
- :  - 字面匹配": "
- (.*) - 第二个捕获组，匹配任意字符（动作参数）
- $ - 行的结束

In [ ]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python正则表达式来选择Action



## 主查询函数

`query` 函数实现了一个多轮对话循环，其中AI代理可以执行动作并根据观察结果继续对话：

**初始化阶段：**
- 创建一个 `Agent` 实例，使用预定义的提示词
- 设置循环计数器和初始问题

**对话循环：**
每一轮循环中，系统会：
1. 调用AI代理生成响应
2. 打印响应内容
3. 使用正则表达式解析响应中的动作指令

**动作解析和执行：**

```python
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
```



这行代码通过列表推导式处理AI响应：
- 将响应按行分割
- 对每行尝试匹配动作模式
- 只保留匹配成功的行

**动作验证和执行：**
如果找到动作指令，系统会：
- 提取动作名称和参数：`action, action_input = actions[0].groups()`
- 验证动作是否在已知动作列表中
- 执行动作并获得观察结果
- 将观察结果作为下一轮的输入

## 关键设计考虑

**错误处理：** 代码包含对未知动作的异常处理，这是一个重要的安全机制。

**循环控制：** `max_turns` 参数防止无限循环，这在AI系统中很重要。

**状态管理：** 通过 `next_prompt` 变量维护对话状态，使AI能够基于之前的观察结果继续推理。

这种模式常用于实现"思考-行动-观察"的AI代理架构，允许AI不仅生成文本响应，还能执行实际的工具调用并根据结果调整后续行为。

In [ ]:

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\xuhao\AppData\Local\Temp\ipykernel_22096\765573690.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python正则表达式来选择Action


In [37]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights for both dogs. I will add 37 lbs (Border Collie) and 20 lbs (Scottish Terrier) to get their combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.
